##### Compare our code with Dr. Adams solution

In [1]:
# import pickle file and print the content
import pickle

# variableAndQmatrix.pickle
with open('variableAndQmatrix.pickle', 'rb') as f:
    data = pickle.load(f)


In [2]:
import numpy as np
# get the determinant of each Q matrix
for variable, Qmatrix in data.items():
    print(variable, sep='\n')
    print('Determinant:', (np.linalg.det(Qmatrix), 2))


EP_DISABL
Determinant: (array([0.]), 2)
EP_PCI
Determinant: (array([0.]), 2)
EP_LIMENG
Determinant: (array([-1.17615891e-10]), 2)
EP_CROWD
Determinant: (array([0.]), 2)
EP_UNINSUR
Determinant: (array([3.91085014e-15]), 2)


In [3]:
Qmatrix = data['EP_LIMENG'][0]

In [4]:
# ger the shape of the df
print(Qmatrix.shape)

(16, 16)


##### Dr. Adams code

In [5]:
import scipy as sp
from scipy.linalg import solve
from scipy.sparse.linalg import spsolve

In [6]:
# Qmatrix dataframe to numpy array
Qmatrix_ = Qmatrix.to_numpy()

In [7]:
# get the determinant of the Q matrix
det = np.linalg.det(Qmatrix_)  
print('Determinant:', det)


Determinant: -1.1761589105402491e-10


In [8]:
n = Qmatrix_.shape[0]
n

16

In [9]:
# Add jitter to the diagonal of 
# In the context of the provided code, "jittered" refers to the process of adding 
# small random variations to the diagonal elements of the matrix Qmatrix_. 
# This is achieved by multiplying a sparse diagonal matrix (constructed with ones along the main diagonal)
# by the maximum value along the diagonal of Qmatrix_ and the square root of the machine epsilon.

Q_jitter = Qmatrix_ +  sp.sparse.diags(np.ones(n)) * max(Qmatrix_.diagonal()) *np.sqrt(

    np.finfo(np.float64).eps

)

without jitter part the scaling factor changes for all variables(even ones with determinant non zero)

In [11]:
# get the determinant of the Q matrix
det = np.linalg.det(Q_jitter)  
print('Determinant:', det)

Determinant: -1.1761589105402491e-10


In [12]:
# inverse of precision (Q) is cov

Q_perturbed = sp.sparse.csc_array(Q_jitter)
b = sp.sparse.identity(n, format='csc')
sigma = spsolve(Q_perturbed, b)

In [13]:
# V \in Null(Q)

V = np.ones(n)  # from pg. 6
W = sigma @ V.T  # \Sigma * B in 3.17
Q_inv = sigma - np.outer(W * solve(V @ W, np.ones(1)), W.T)

In [14]:
# grabbing diag of cov gives var and

# arithmetic mean in log-space becomes geometric mean after exp
scaling = np.exp(np.mean(np.log(np.diag(Q_inv))))
print(scaling)

0.7354405665541911
